In [1]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import pandas as pd

In [2]:
# get list of rsid-s for scraping from the file
with open('data/list-test.txt') as fh:
    ls_rsid = [rsid.strip() for rsid in fh.readlines()]
    
print('Number of rsid to process:', len(ls_rsid))

Number of rsid to process: 5


In [3]:
df = pd.DataFrame()

In [4]:
def handle_rsid(rsid):
    """
    The function gets 1 rsid (as string or integer), mades query in usual way to NCBI server,
    extracts some data from the code and packs them to Pandas Series, that is returned from the function.
    The functin is not ideal in terms of defence against server error, and sometimes can break down.
    But if server response is good, it can process a thousand of rsid-s.    
    """
    
    # get html-code of the page
    url = 'https://www.ncbi.nlm.nih.gov/snp/' + str(rsid)
    
    # Made response to the server. If there is a problem, try again.
    response = requests.get(url)
    while response.status_code != 200:
        print(f'WARNING: rsid = {rsid}, response_code = {response.status_code}')
        sleep(3)
        response = requests.get(url)
    
    # parse response-object to BeautifulSoup-object
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    
    # preliminaty extract from html-code part of code with necessary values by specified tags and their attributes
    ls_dl = soup.find('div', attrs={'class': 'summary-box usa-grid-full'}).find_all('dl', attrs={'class': 'usa-width-one-half'})

    # extract some values and place them into variable (left half of data block on the page)
    ls_dt = ls_dl[0].find_all('dt')
    ls_dd = ls_dl[0].find_all('dd')
    for el_dt, el_dd in zip(ls_dt, ls_dd):
        el_dt = el_dt.text.strip()
        if el_dt == 'Organism':
            organism = el_dd.text.strip()
        elif el_dt == 'Position':
            ls_span = el_dd.find_all('span')
            pos = ls_span[0].text.strip()
            ref_genome = ls_span[1].text.strip(" ( )")
        elif el_dt == 'Alleles':
            alleles = el_dd.text.strip()
            ls_allele_pairs = alleles.split(' / ')
            ref_allele = ''
            alt_alleles = ''
            for pair in ls_allele_pairs:
                if len(pair) != 3:
                    print(f"WARNING: rsid = {rsid}: problem with splitting of allele string, length of pair is {len(pair)} instead of expected 3")
                    print(f"Alleles: {pair[:30]}{'…' if len(pair)>30 else ''}")
                    ref_allele = '-'
                    alt_alleles = '-'
                else:
                    try:
                        al_1, al_2 = pair.split('>')
                    except ValueError as ex:
                        print(f'CAUGHT ERROR: rsid = {rsid}: {ex}')
                        ref_allele = '-'
                        alt_alleles = '-'
                        break
                    else:
                        if ref_allele == '':
                            ref_allele = al_1
                        elif al_1 != ref_allele:
                            ref_allele += ', ' + al_1
                            print(f'For rsid = {rsid}, there are more than one reference allele')
                            # raise ValueError(f'For rsid = {rsid}, there is more than one reference allele')

                        if alt_alleles == '':
                            alt_alleles = al_2
                        else:
                            alt_alleles += ', ' + al_2

        elif el_dt == 'Variation Type':
            variation_type = el_dd.text.lstrip().split('\n')[0]
    
    # extract some values and place them into variable (right half of data block on the page)
    ls_dt = ls_dl[1].find_all('dt')
    ls_dd = ls_dl[1].find_all('dd')
    for el_dt, el_dd in zip(ls_dt, ls_dd):
        el_dt = el_dt.text.strip()
        if el_dt == 'Clinical Significance':
            clinical_significance = el_dd.text.strip()
        elif el_dt == 'Gene : Consequence':
            gene_consequence = '; '.join(map(lambda tag:tag.text.strip(), el_dd.findAll() ) )    #el_dd.text.strip()
        elif el_dt == 'Publications':
            publications = ' '.join(st.strip() for st in el_dd.text.strip().split('\n'))

    # pack variables into Pandas series
    ser = pd.Series({
        'rsid': rsid,
        'organism': organism,
        'position': pos,
        'ref_genome': ref_genome,
        'ref_allele': ref_allele,
        'alt_alleles': alt_alleles,
        'variation_type': variation_type,
        'clinical_significance': clinical_significance,
        'gene_consequence': gene_consequence,
        'publications': publications
    })
    
    return ser


# print header for preview inside this Jupyter notebook
print("  #   rsid         ref_allele    alt_alleles")

start_processing = 0
for i, rsid in enumerate(ls_rsid[start_processing:], start=start_processing):
    
    # optimization so that we don't send request with the same rsid to NCBI several times
    if not df.empty and (rsid in df.rsid.to_list()):
        ser = df[df.rsid == rsid].iloc[0]
    else:
        ser = handle_rsid(rsid)
        
    ser.name = i
    df = df.append(ser)
    
    # preview
    print(f"{i:3}   {rsid:11}   {ser.ref_allele}             {ser.alt_alleles}")
    
    sleep(3)

df

  #   rsid         ref_allele    alt_alleles
  0   rs5128        G             C
  1   rs7412        C             T
Alleles: ins(T)11GAGACGGAGTCTCGCTCTGTCG…
  2   rs1799752     -             -
  3   rs186838828   T             A
  4   rs191667986   T             C


,rsid,organism,position,ref_genome,ref_allele,alt_alleles,variation_type,clinical_significance,gene_consequence,publications
0,rs5128,Homo sapiens,chr11:116832924,GRCh38.p13,G,C,SNV,Not Reported in ClinVar,APOC3 : 3 Prime UTR Variant,47 citations
1,rs7412,Homo sapiens,chr19:44908822,GRCh38.p13,C,T,SNV,Reported in ClinVar,APOE : Missense Variant,386 citations
2,rs1799752,Homo sapiens,chr17:63488530-63488543,GRCh38.p13,-,-,Indel,Reported in ClinVar,ACE : Intron Variant,74 citations
3,rs186838828,Homo sapiens,chr2:166195246,GRCh38.p13,T,A,SNV,Reported in ClinVar,SCN1A-AS1 : Intron Variant; SCN9A : 3 Prime UT...,0 citations
4,rs191667986,Homo sapiens,chr2:166195355,GRCh38.p13,T,C,SNV,Reported in ClinVar,SCN1A-AS1 : Intron Variant; SCN9A : 3 Prime UT...,0 citations


In [5]:
df.index.name='#'
df.to_csv("output/data_NCBI.csv")